# Deploy DL Model on Web & Mobile Using Pyotrch
---
## Milestone 1: Build an image classifier
## Partial Solutions Notebook
---
#### Date updated:  12-September-2021
#### Author:  Nidhin Pattaniyil & Reshama Shaikh


In [1]:
%matplotlib widget

In [2]:
!ls

01_training.ipynb	     ant_transfer_learning_tutorial.ipynb
01_training__output.ipynb    artifacts
01_training_raw_food.ipynb   data
02_torch_optimization.ipynb  lightning_logs
Untitled.ipynb		     milestone1_partial_solution_3classes.ipynb
Untitled1.ipynb		     requirements.txt


## Imports

In [3]:
# from datetime import date
# from datetime import datetime
import datetime
import plotly.figure_factory as ff
import wandb
from matplotlib import pyplot as plt
import sklearn
import pytorch_lightning as pl
import flash
import flash.image
from flash.core.data.utils import download_data
# from flash.image import ImageClassificationData, ImageClassifier
import os
import glob
import pandas as pd
import sklearn.model_selection
#from sklearn.model_selection import train_test_split 
#from flash.image.classification.adapters import TRAINING_STRATEGIES
# from flash.image.classification.backbones import IMAGE_CLASSIFIER_BACKBONES
# from flash.core.classification import Probabilities
import numpy as np
import torch
from torchvision import transforms as T
import flash
from flash.core.data.data_source import DefaultDataKeys
from flash.core.data.transforms import ApplyToKeys, merge_transforms
# from flash.image import ImageClassificationData, ImageClassifier
# from flash.image.classification.transforms import default_transforms
import flash.core.integrations.fiftyone
import flash.core.classification 
import fiftyone as fo
import warnings
import itertools
import timeit
import json
import torchinfo

In [4]:
pl.seed_everything(99)

Global seed set to 99


99

In [5]:
flash.__version__

'0.5.2'

## Parameters


In [6]:
model_name = 'resnet34'

In [7]:
# Parameters
model = "resnet34"


## Helper code

In [8]:
def create_fiftyone_dataset(df:pd.DataFrame, name="test_df"):
    samples = []
    
    if fo.dataset_exists(name):
        fo.delete_dataset(name)
    for record in df.to_dict(orient='records'):
        samples.append(
            fo.Sample(
                filepath=record['file_path'],
                ground_truth=fo.Classification(label=record['label_name']),
            )
        )
    dataset = fo.Dataset(name)
    dataset.add_samples(samples)
    return dataset

In [9]:
!ls

01_training.ipynb	     ant_transfer_learning_tutorial.ipynb
01_training__output.ipynb    artifacts
01_training_raw_food.ipynb   data
02_torch_optimization.ipynb  lightning_logs
Untitled.ipynb		     milestone1_partial_solution_3classes.ipynb
Untitled1.ipynb		     requirements.txt


In [10]:


current_date = datetime.date.today()
print("Today's date:", current_date)

Today's date: 2021-11-08


In [11]:
now1 = datetime.datetime.now()

start_time = now1.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 10:43:05


In [12]:
#!pip install lightning-flash==0.5.0

In [13]:
# get the backbones available for ImageClassifier
backbones = flash.image.ImageClassifier.available_backbones()

# print the backbones
#print(backbones)

In [14]:
warnings.filterwarnings("ignore", message="Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.")

In [15]:

# https://lp-prod-resources.s3-us-west-2.amazonaws.com/other/Deploying+a+Deep+Learning+Model+on+Web+and+Mobile+Applications+Using+TensorFlow/Food+101+-+Data+Subset.zip

In [16]:
#download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", 'data/')


## Prepare dataset

In [17]:
data_dir = "data/food-101-subset/images/"

In [18]:
# files = glob.glob(f"{data_dir}/*/*.jpg")
# df = pd.DataFrame({'file_path':files})
# df['label_name'] = df['file_path'].apply(lambda x:x.split("/")[-2])
# df.head()

In [19]:
df = pd.read_csv("data/food-101/meta/train.txt",header=None,names=['file_path'])
df['label_name'] = df['file_path'].apply(lambda x:x.split("/")[-2])

df['file_path'] = "data/food-101/images/" + df["file_path"]+".jpg"




In [20]:
df.head()

,file_path,label_name
0,data/food-101/images/apple_pie/1005649.jpg,apple_pie
1,data/food-101/images/apple_pie/1014775.jpg,apple_pie
2,data/food-101/images/apple_pie/1026328.jpg,apple_pie
3,data/food-101/images/apple_pie/1028787.jpg,apple_pie
4,data/food-101/images/apple_pie/1043283.jpg,apple_pie


In [21]:
label_encoder = sklearn.preprocessing.LabelEncoder()


In [22]:
label_encoder.fit(df['label_name'])


LabelEncoder()

In [23]:
df['label'] = label_encoder.transform(df['label_name'])


In [24]:
df_eval = pd.read_csv("data/food-101/meta/test.txt",header=None,names=['file_path'])
df_eval['label_name'] = df_eval['file_path'].apply(lambda x:x.split("/")[-2])
df_eval['file_path'] = "data/food-101/images/" + df_eval["file_path"]+".jpg"
df_eval['label'] = label_encoder.transform(df_eval['label_name'])


In [25]:
#!ls data/food-101/images

In [26]:
df_eval.head()

,file_path,label_name,label
0,data/food-101/images/apple_pie/1011328.jpg,apple_pie,0
1,data/food-101/images/apple_pie/101251.jpg,apple_pie,0
2,data/food-101/images/apple_pie/1034399.jpg,apple_pie,0
3,data/food-101/images/apple_pie/103801.jpg,apple_pie,0
4,data/food-101/images/apple_pie/1038694.jpg,apple_pie,0


In [27]:
# split the dataset; 85% for training and 15 % for testing
df_train, df_test = sklearn.model_selection.train_test_split(df, test_size=0.15, stratify=df['label'],shuffle = True) 

# split the training dataset: 80% for actual training and 20% for validation
df_train, df_val = sklearn.model_selection.train_test_split(df_train, test_size=0.20, random_state=1 , stratify=df_train['label'])

In [28]:
print ({"train": len(df_train), "test":len(df_test), "val":len(df_val) ,"eval": len(df_eval)  })

{'train': 51509, 'test': 11363, 'val': 12878, 'eval': 25250}


In [29]:
default_transforms = flash.image.classification.transforms.default_transforms((224, 224))
default_transforms

{'to_tensor_transform': Sequential(
   (0): ApplyToKeys(keys=<DefaultDataKeys.INPUT: 'input'>, transform=FuncModule())
   (1): ApplyToKeys(keys=<DefaultDataKeys.TARGET: 'target'>, transform=FuncModule())
 ),
 'post_tensor_transform': ApplyToKeys(keys=<DefaultDataKeys.INPUT: 'input'>, transform=Resize()),
 'collate': <function flash.core.data.transforms.kornia_collate(samples: Sequence[Dict[str, Any]]) -> Dict[str, Any]>,
 'per_batch_transform_on_device': ApplyToKeys(keys=<DefaultDataKeys.INPUT: 'input'>, transform=Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]), p=1.0, p_batch=1.0, same_on_batch=True, return_transform=False))}

In [30]:
post_tensor_transform = ApplyToKeys(
    DefaultDataKeys.INPUT,
    T.Compose([T.RandomHorizontalFlip(), T.ColorJitter(), T.RandomAutocontrast(), T.RandomPerspective()]),
)

new_transforms = merge_transforms(default_transforms, {"post_tensor_transform": post_tensor_transform})

In [31]:
# datamodule = ImageClassificationData.from_data_frame(
#     "file_path",
#     "label",
#     train_data_frame=df_train,
#     val_data_frame=df_val,
#     test_data_frame=df_test,
#     train_transform=new_transforms
# )

In [32]:
fo.delete_non_persistent_datasets()

In [33]:
train_dataset = create_fiftyone_dataset (df_train,"train")
val_dataset = create_fiftyone_dataset (df_val,"validation") 
test_dataset = create_fiftyone_dataset (df_test,"test")
eval_dataset = create_fiftyone_dataset (df_eval,"eval")




 100% |█████████████| 51509/51509 [40.4s elapsed, 0s remaining, 1.5K samples/s]      
 100% |█████████████| 12878/12878 [10.2s elapsed, 0s remaining, 1.3K samples/s]     
 100% |█████████████| 11363/11363 [8.8s elapsed, 0s remaining, 1.8K samples/s]      
 100% |█████████████| 25250/25250 [21.1s elapsed, 0s remaining, 971.8 samples/s]      


In [34]:
datamodule = flash.image.ImageClassificationData.from_fiftyone(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    train_transform=new_transforms,
)

In [35]:
train_dataset

Name:        train
Media type:  image
Num samples: 51509
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)

In [36]:
datamodule.num_classes


101

## Model

In [37]:
model_metrics = {}

In [38]:
sample_image = "data/food-101-subset/images/apple_pie/1005649.jpg"
strategy="dp"


In [39]:
sample_input_size = (1, 3, 224, 224)


In [40]:
def time_inference(model,device='cpu'):
    model = model.eval()

    with torch.no_grad():
        model.to(device)
        predictions = model.predict(sample_image)


In [41]:
def train_model(model_name:str, epochs = 2, unfreeze_epoch = 5 ,num_iterations=10, df_eval=df_eval  ):
    model = flash.image.ImageClassifier(num_classes=datamodule.num_classes,backbone=model_name)
    print (f"Model Spec: {model_name}")
    print(model)
    
    
    
    print ("Model Summary")
    
    model_summary = torchinfo.summary(model, input_size=sample_input_size, verbose=0)
    print (model_summary)
    
    
    
            
    
    trainer = flash.Trainer(max_epochs=epochs, gpus=torch.cuda.device_count() , strategy=strategy)

    
    
    trainer.finetune(model, datamodule=datamodule
                 , strategy=flash.core.finetuning.FreezeUnfreeze(unfreeze_epoch=unfreeze_epoch) )
    
    
    metrics_test = trainer.test(model, datamodule=datamodule)
    print (metrics_test)

    
    metrics_val =trainer.validate(model, datamodule=datamodule)
    print(metrics_val)
    
    artifact_model_path = f"artifacts/model_{model_name}.pt"
    trainer.save_checkpoint(artifact_model_path)

    model = flash.image.ImageClassifier.load_from_checkpoint(artifact_model_path)
    
    time_gpu = timeit.timeit(lambda: time_inference (model,device='cuda:0') ,number=num_iterations   ) 
    time_cpu = timeit.timeit(lambda: time_inference (model,device='cpu') ,number=num_iterations   ) 
    
    
    # eval dataset accuracy
    eval_dataset = create_fiftyone_dataset (df_eval,"eval")
    datamodule_eval = flash.image.ImageClassificationData.from_fiftyone(predict_dataset=eval_dataset)
    predictions = trainer.predict(model, datamodule=datamodule_eval) # datamodule_predict
    predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches
    y_proba = predictions
    y_pred = np.argmax(y_proba, axis=1)
    y_true = df_test['label']

    eval_accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    
    
    
    res = {**metrics_test[0], **metrics_val[0]}
    res['time_cpu'] = time_cpu
    res['time_gpu'] = time_gpu
    res['eval_accuracy'] = eval_accuracy
    
    res['params_total'] = model_summary.total_params
    res['params_trainable'] = model_summary.trainable_params
    


    model_metrics[model_name] = res
    
    
    

    return model
    
    

In [42]:
max_epochs = 15
unfreeze_epoch = 5

In [43]:
trainer = flash.Trainer(max_epochs=max_epochs, gpus=torch.cuda.device_count(), strategy=strategy  )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [44]:
model_name = 'vgg19'
model = train_model(model_name=model_name, epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)


Using 'vgg19' provided by PyTorch/torchvision (https://github.com/pytorch/vision).


Model Spec: vgg19
ImageClassifier(
  (train_metrics): ModuleDict(
    (accuracy): Accuracy()
  )
  (val_metrics): ModuleDict(
    (accuracy): Accuracy()
  )
  (test_metrics): ModuleDict(
    (accuracy): Accuracy()
  )
  (adapter): DefaultAdapter(
    (backbone): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/opt/conda/envs/pytorch/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:91: LightningDeprecationWarning: Method `on_train_dataloader` is deprecated in v1.5.0 and will be removed in v1.7.0. Please use `train_dataloader()` directly.
  "Method `on_train_dataloader` is deprecated in v1.5.0 and will be removed in v1.7.0."
/opt/conda/envs/pytorch/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:126: LightningDeprecationWarning: Method `on_val_dataloader` is deprecated in v1.5.0 and will be removed in v1.7.0. Please use `val_dataloader()` directly.
  "Method `on_val_dataloader` is deprecated in v1.5.0 and will be removed in v1.7.0."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDic

Layer (type:depth-idx)                   Output Shape              Param #
ImageClassifier                          --                        --
├─DefaultAdapter: 1                      --                        --
│    └─Sequential: 2-1                   [1, 512, 7, 7]            --
│    │    └─Conv2d: 3-1                  [1, 64, 224, 224]         1,792
│    │    └─ReLU: 3-2                    [1, 64, 224, 224]         --
│    │    └─Conv2d: 3-3                  [1, 64, 224, 224]         36,928
│    │    └─ReLU: 3-4                    [1, 64, 224, 224]         --
│    │    └─MaxPool2d: 3-5               [1, 64, 112, 112]         --
│    │    └─Conv2d: 3-6                  [1, 128, 112, 112]        73,856
│    │    └─ReLU: 3-7                    [1, 128, 112, 112]        --
│    │    └─Conv2d: 3-8                  [1, 128, 112, 112]        147,584
│    │    └─ReLU: 3-9                    [1, 128, 112, 112]        --
│    │    └─MaxPool2d: 3-10              [1, 128, 56, 56]          --

Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/envs/pytorch/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/opt/conda/envs/pytorch/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/data.py:57: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 4. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
Global seed set to 99
/opt/conda/envs/pytorch/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_da

Training: 0it [00:00, ?it/s]

In [ ]:
train_model(model_name='resnet50', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
train_model(model_name='mobilenet_v2', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
train_model(model_name='mobilenetv3_small_100', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
model_metrics

In [ ]:
with open("artifacts/model_metrics.json") as f:
    payload = json.dumps(model_metrics)
    f.write(payload)


In [ ]:
df_train.to_parquet("artifacts/data/df_train.parquet")
df_test.to_parquet("artifacts/data/df_test.parquet")
df_val.to_parquet("artifacts/data/df_val.parquet")
df_eval.to_parquet("artifacts/data/adf_eval.parquet")

## Best Model

In [ ]:
model_name = "mobilenet_v2"
artifact_model_path = f"artifacts/model_{model_name}.pt"

In [ ]:
model = flash.image.ImageClassifier.load_from_checkpoint(artifact_model_path)
model

In [ ]:
model_summary = torchinfo.summary(model, input_size=sample_input_size, verbose=0)
model_summary

## Predict on Single

In [ ]:
model.serializer = flash.core.classification.Probabilities()

In [ ]:
predictions = model.predict("data/food-101-subset/images/apple_pie/1005649.jpg")
predictions
#label_encoder.inverse_transform(predictions)

In [ ]:
predictions[0]

In [ ]:
sorted ( zip (label_encoder.classes_, predictions[0]), key= lambda x:x[1] ,reverse =True)

In [ ]:
# datamodule_predict = ImageClassificationData.from_data_frame(
#     "file_path",
#     "label",
#     test_data_frame=df_test,
#     predict_transform=default_transforms
# )
# datamodule_predict

## Predict on dataset

In [ ]:
#?flash.core.classification.PredsClassificationSerializer

In [ ]:
# model.serializer = flash.core.classification.PredsClassificationSerializer()
# fo_predict = create_fiftyone_dataset(df_test,"predict")
# datamodule_predict = ImageClassificationData.from_fiftyone(fo_predict)


In [ ]:
model.serializer = flash.core.classification.FiftyOneLabels(return_filepath=False, labels=label_encoder.classes_)  # output FiftyOne format
fo_predict = create_fiftyone_dataset(df_eval,"predict")
datamodule_predict = flash.image.ImageClassificationData.from_fiftyone(predict_dataset=fo_predict)


In [ ]:
predictions = trainer.predict(model, datamodule=datamodule_predict) # datamodule_predict
predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches

fo_predict.set_values("predictions", predictions)

In [ ]:
predictions[0]

In [ ]:
results = fo_predict.evaluate_classifications("predictions", gt_field="ground_truth", eval_key="eval")
results.print_report()

In [ ]:
plot = results.plot_confusion_matrix(classes=label_encoder.classes_)
plot.show()

In [ ]:
# 8 Visualize results in the App
session = fo.launch_app(fo_predict, auto=False)



In [ ]:
session.open_tab()


In [ ]:
!ip a

In [ ]:
# Plot confusion matrix
plot = results.plot_confusion_matrix(classes=label_encoder.classes_)
plot.show(height=600)



In [ ]:
# Connect to session
#session.plots.attach(plot)

In [ ]:
# # Optional: block execution until App is closed
# session.wait()

## Prediction using datamodule

In [ ]:
model.serializer = flash.core.classification.Probabilities()

predictions = trainer.predict(model,datamodule=datamodule_predict)
predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches


In [ ]:
# y_proba = model.predict(list ( df_test['file_path'] ) )
# y_pred = np.argmax(y_proba, axis=1)
# y_true = df_test['label']

In [ ]:
y_proba = predictions
y_pred = np.argmax(y_proba, axis=1)
y_true = df_test['label']

In [ ]:
# https://stackoverflow.com/questions/60860121/plotly-how-to-make-an-annotated-confusion-matrix-using-a-heatmap
fig, ax = plt.subplots(figsize=(15,15))
cm_labels = label_encoder.classes_
disp = sklearn.metrics.ConfusionMatrixDisplay.from_predictions(
    y_true=y_true, y_pred = y_pred ,
    #normalize = 'true',
    display_labels=cm_labels,
    xticks_rotation='vertical',
    ax=ax
)
disp

## Model inference

In [ ]:
sample_image = "data/food-101-subset/images/apple_pie/1005649.jpg"

In [ ]:
model.eval()

In [ ]:
%%timeit -r 9 -n 5
time_inference (model,device='cuda:0')


In [ ]:
%%timeit -r 9 -n 5
time_inference (model,device='cpu')


# Get Model Size

In [ ]:
! du -h artifacts

In [ ]:
!ls artifacts/ -lah

# Compare Models

- Dataset:  Food
- Classes: 3
- Total images: 3000
- Batch size: 32

| Model  | Trainable Params| Non-trainable Params | Time (hh:mm:ss) [a]| Accuracy | Epochs | Model Size [b]
|---|---:|---:|---:|---:| ---:|---:|
| VGG19| 528,387   | 20,024,384 | 00:00:48 | 96.8% | 3 | 83M
| ResNet50   | xxx   | xxx |  xx:xx:xx | xx.x% | 3 | xxM
| MobileNetV2  | xxx  | xxx |   xx:xx:xx | xx.x% | 3 | xxM
| ResNet50_ft [c]  | xxx  | xxx | xx:xx:xx   | xx.x% | xx | xxM

NOTES:  
- [a] If wall clock time < CPU time, then you're executing a program in parallel.
- [b] model size is size of output file
- [c] ft = fine-tuned; time 2min 31s; 3min 59s; epochs 9+5


# Download Assets
download model and classes.json

In [ ]:
#!zip -r artifacts/artifacts.zip {PROJECT_NAME}/artifacts/

In [ ]:
# from google.colab import files
# files.download(str(str(PROJECT_NAME)+"/artifacts/artifacts.zip"))